In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
import pandas as pd

subspecies = pd.read_csv("../Data/mgSs.coverage.stats2.csv").rename(columns={"Unnamed: 0" : "subspecies"})
subspecies.head()

,subspecies,as.factor(sample_cluster),n,mean,sd,min,Q1,median,Q3,max
0,Aerococcus_christensenii.1,1,283,23224,25864,2073,8936,15558,26066,212022
1,Aerococcus_christensenii.2,2,196,9875,11638,1807,4122,6345,9934,97899
2,Aerococcus_christensenii.3,3,137,54092,71171,4592,15859,31012,63044,459226
3,Aerococcus_christensenii.4,4,86,15893,13405,2192,6845,11511,19137,65486
4,Aerococcus_christensenii.5,5,77,27587,54815,1842,4651,11428,30731,451000


In [2]:
subspecies = pd.read_csv("../Data/mgSs.coverage.stats2.csv").rename(columns={"Unnamed: 0" : "subspecies"})
subspecies = subspecies['subspecies']
taxaKey = pd.read_csv("/Users/amaros/Library/CloudStorage/OneDrive-UniversityofMarylandSchoolofMedicine/mgss2/SOURCE_DATA/VIRGO2_taxaKey_all_gardnerella.csv")
def lighter_colors(base_color, num_colors):
    # Convert the base color to its RGB components
    r = int(base_color[1:3], 16)
    g = int(base_color[3:5], 16)
    b = int(base_color[5:7], 16)

    # Calculate the step values for degrading
    r_step = (255 - r) / num_colors
    g_step = (255 - g) / num_colors
    b_step = (255 - b) / num_colors

    # Generate the progressively lighter colors
    lighter_colors = []
    for i in range(num_colors):
        new_r = int(r + i * r_step)
        new_g = int(g + i * g_step)
        new_b = int(b + i * b_step)
        lighter_colors.append(f'#{new_r:02X}{new_g:02X}{new_b:02X}')

    return lighter_colors

species = subspecies.apply(lambda x : x.split(".")[0])
couleur = []
for i,j in zip(list(species.value_counts().sort_index().index), species.value_counts().sort_index()) :
    if i in taxaKey['Taxa'].values:
        a = taxaKey[taxaKey['Taxa'] == i]['Color'].values[0]
        couleur.append(lighter_colors(a,j))
    else :
        couleur.append(['#8c8c8c']*j)

# Original list with single elements and sublists
original_list = couleur
# Create a new list with only the single elements
new_list = [item for sublist in original_list for item in (sublist if isinstance(sublist, list) else [sublist])]

subspecies_with_colors = pd.DataFrame({"Subspecies" : subspecies, "Color" : new_list})
subspecies_with_colors.to_csv("../Data/subspecies_with_colors2.csv", index=False)
# subspecies_with_colors.head()

In [ ]:
# Variation of colors between 2 kind of blues for Gardnerella : this generate 58 variation of blues

def interpolate_color(color1, color2, num_steps):
    # Convert hex colors to RGB tuples
    color1 = tuple(int(color1[i:i+2], 16) for i in (1, 3, 5))
    color2 = tuple(int(color2[i:i+2], 16) for i in (1, 3, 5))

    color_variations = []

    for step in range(num_steps):
        r = int(color1[0] + (color2[0] - color1[0]) * step / (num_steps - 1))
        g = int(color1[1] + (color2[1] - color1[1]) * step / (num_steps - 1))
        b = int(color1[2] + (color2[2] - color1[2]) * step / (num_steps - 1))

        # Convert RGB to hex
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)

        color_variations.append(hex_color)

    return color_variations

# Example usage:
color1 = "#221886"  # Blue
color2 = "#9A98BF"  # Cyan (light blue)
num_steps = 58  # Number of variations

variations = interpolate_color(color1, color2, num_steps)
print(variations)


In [2]:
mgcsts_samples = pd.read_csv("../Data/mgCSTs.samples.df2.csv")
mgCSTs_sort = pd.read_csv("../Data/mgCSTs.sort.df2.csv")
subspecies_with_colors = pd.read_csv("../Data/subspecies_with_colors2.csv")
data = mgcsts_samples
data2 = mgCSTs_sort
# deepsplit = 4
# mincluster = 30

# data = mgcsts_samples[(mgcsts_samples['deepSplit'] == deepsplit) & (mgcsts_samples['minClusterSize'] == mincluster)]
# data = data.reset_index(drop = True)

# data2 = mgCSTs_sort[(mgCSTs_sort['deepSplit'] == deepsplit) & (mgCSTs_sort['minClusterSize'] == mincluster)]
# data2 = data2.reset_index(drop = True)


# count_sample = []
# for element in data2['dtc'].values :
#     count_sample.append(data.groupby(['dtc']).count()['sampleID'][element])
# data2['count_sample'] = count_sample

color_sort = []
for element in data2['domTaxa'].values :
    a = subspecies_with_colors[subspecies_with_colors['Subspecies'].apply(lambda x : x.replace(".", "_")) == element]['Color'].values
    if a.size > 0 :
        color_sort.append(a[0])
    else :
        color_sort.append("#8c8c8c")
data2['color_mgCST'] = color_sort

data2.to_csv("../Data/mgCST_sort_color2.csv", index=False)

## To add the color of the Dominant taxa IN THE SAMPLE - too confusing

# color_samples = []
# for element in data['domTaxa'].values :
#     a = subspecies_with_colors[subspecies_with_colors['Subspecies'].apply(lambda x : x.replace(".", "_")) == element]['Color'].values
#     if a.size > 0 :
#         color_samples.append(a[0])
#     else :
#         color_samples.append("#8c8c8c")
# data['color_domTaxa'] = color_samples
# data.to_csv("/Users/amaros/Desktop/mgss2/streamlit_app/Data/mgCST_samples_color.csv", index=False)
